In [1]:
from joblib import Parallel, delayed
import multiprocessing

In [2]:
num_cores = multiprocessing.cpu_count()
print(num_cores)

8


In [3]:
####Simple Logistic regression (without parallelisation)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
df = pd.read_csv(r"creditcard.csv", dtype='unicode')

In [6]:
df["Time"] = pd.to_numeric(df["Time"], downcast="float")
df["V1"] = pd.to_numeric(df["V1"], downcast="float")
df["V2"] = pd.to_numeric(df["V2"], downcast="float")
df["V3"] = pd.to_numeric(df["V3"], downcast="float")
df["V4"] = pd.to_numeric(df["V4"], downcast="float")
df["V5"] = pd.to_numeric(df["V5"], downcast="float")
df["V6"] = pd.to_numeric(df["V6"], downcast="float")
df["V7"] = pd.to_numeric(df["V7"], downcast="float")
df["V8"] = pd.to_numeric(df["V8"], downcast="float")
df["V9"] = pd.to_numeric(df["V9"], downcast="float")
df["V10"] = pd.to_numeric(df["V10"], downcast="float")
df["V11"] = pd.to_numeric(df["V11"], downcast="float")
df["V12"] = pd.to_numeric(df["V12"], downcast="float")
df["V13"] = pd.to_numeric(df["V13"], downcast="float")
df["V14"] = pd.to_numeric(df["V14"], downcast="float")
df["V15"] = pd.to_numeric(df["V15"], downcast="float")
df["V16"] = pd.to_numeric(df["V16"], downcast="float")
df["V17"] = pd.to_numeric(df["V17"], downcast="float")
df["V18"] = pd.to_numeric(df["V18"], downcast="float")
df["V19"] = pd.to_numeric(df["V19"], downcast="float")
df["V20"] = pd.to_numeric(df["V20"], downcast="float")
df["V21"] = pd.to_numeric(df["V21"], downcast="float")
df["V22"] = pd.to_numeric(df["V22"], downcast="float")
df["V23"] = pd.to_numeric(df["V23"], downcast="float")
df["V24"] = pd.to_numeric(df["V24"], downcast="float")
df["V25"] = pd.to_numeric(df["V25"], downcast="float")
df["V26"] = pd.to_numeric(df["V26"], downcast="float")
df["V27"] = pd.to_numeric(df["V27"], downcast="float")
df["V28"] = pd.to_numeric(df["V28"], downcast="float")

In [7]:
df["Amount"] = pd.to_numeric(df["Amount"], downcast="float")
df["Class"] = pd.to_numeric(df["Class"], downcast="float")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.619995,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.690000,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.660004,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.500000,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798279,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.989998,0.0


In [9]:
X = df.iloc[:, :-1]

In [10]:
y = df.iloc[:, -1]

In [11]:
print(np.c_[np.ones((X.shape[0],1)), X])

[[ 1.00000000e+00  0.00000000e+00 -1.35980713e+00 ...  1.33558378e-01
  -2.10530534e-02  1.49619995e+02]
 [ 1.00000000e+00  0.00000000e+00  1.19185710e+00 ... -8.98309890e-03
   1.47241689e-02  2.69000006e+00]
 [ 1.00000000e+00  1.00000000e+00 -1.35835409e+00 ... -5.53527959e-02
  -5.97518422e-02  3.78660004e+02]
 ...
 [ 1.00000000e+00  1.72788000e+05  1.91956496e+00 ...  4.45477199e-03
  -2.65608281e-02  6.78799973e+01]
 [ 1.00000000e+00  1.72788000e+05 -2.40440056e-01 ...  1.08820736e-01
   1.04532823e-01  1.00000000e+01]
 [ 1.00000000e+00  1.72792000e+05 -5.33412516e-01 ... -2.41530873e-03
   1.36489142e-02  2.17000000e+02]]


In [12]:
X = np.c_[np.ones((X.shape[0], 1)), X]
print(X)
y = y[:, np.newaxis]
theta = np.zeros((X.shape[1], 1))

[[ 1.00000000e+00  0.00000000e+00 -1.35980713e+00 ...  1.33558378e-01
  -2.10530534e-02  1.49619995e+02]
 [ 1.00000000e+00  0.00000000e+00  1.19185710e+00 ... -8.98309890e-03
   1.47241689e-02  2.69000006e+00]
 [ 1.00000000e+00  1.00000000e+00 -1.35835409e+00 ... -5.53527959e-02
  -5.97518422e-02  3.78660004e+02]
 ...
 [ 1.00000000e+00  1.72788000e+05  1.91956496e+00 ...  4.45477199e-03
  -2.65608281e-02  6.78799973e+01]
 [ 1.00000000e+00  1.72788000e+05 -2.40440056e-01 ...  1.08820736e-01
   1.04532823e-01  1.00000000e+01]
 [ 1.00000000e+00  1.72792000e+05 -5.33412516e-01 ... -2.41530873e-03
   1.36489142e-02  2.17000000e+02]]


/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/2100395636.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [13]:
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(theta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, theta)

def probability(theta, x):
    # Returns the probability after passing through sigmoid
    return sigmoid(net_input(theta, x))

In [14]:
def cost_function(theta, x, y):
    # Computes the cost function for all the training samples
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(
        y * np.log(probability(theta, x)) + (1 - y) * np.log(
            1 - probability(theta, x)))
    return total_cost

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta,   x)) - y)

In [15]:
from scipy.optimize import fmin_tnc

In [16]:
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta,
                  fprime=gradient,args=(x, y.flatten()))
    return opt_weights[0]
parameters = fit(X, y, theta)

  NIT   NF   F                       GTG
    0    1  6.931471805599448E-01   2.23421290E+09
tnc: fscale = 2.11562e-05
    1    5  3.435425252098943E-02   8.82919402E+03
tnc: fscale = 0.0106424
/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/2888755855.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/2619135858.py:5: RuntimeWarning: divide by zero encountered in log
  y * np.log(probability(theta, x)) + (1 - y) * np.log(
/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/2619135858.py:5: RuntimeWarning: invalid value encountered in multiply
  y * np.log(probability(theta, x)) + (1 - y) * np.log(
    1   72  3.435425252098943E-02   8.82919402E+03
tnc: Linear search failed


In [17]:
print(parameters)

[-3.72322823e-05 -9.83414348e-05  1.40589558e-05  1.03318863e-06
 -5.19534435e-05 -8.17751331e-06  1.57703812e-05 -7.15822496e-06
  4.21889717e-06 -2.92089941e-06 -2.37382392e-06 -1.17691899e-06
 -1.63907485e-05  5.22273829e-06 -4.95873132e-06 -1.15516932e-05
 -1.25742046e-05 -1.89279070e-06 -8.93874060e-06  4.20055217e-06
  2.19683468e-06 -2.68223587e-06  2.90915408e-06  7.80030169e-06
  2.35407402e-06 -7.98197604e-07 -9.02827965e-06 -1.45480075e-06
 -4.44368100e-08 -1.82747224e-07 -3.46504893e-03]


In [18]:
def predict(x):
    theta = parameters[:, np.newaxis]
    return probability(theta, x)
def accuracy(x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x) >= 
                         probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = np.mean(predicted_classes == actual_classes)
    return accuracy * 100

In [19]:
accuracy(X, y.flatten())

99.82725143693798

### Parallel logistic regression using threads

In [20]:
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(theta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, theta)

def probability(theta, x):
    # Returns the probability after passing through sigmoid
    return sigmoid(net_input(theta, x))

In [21]:
def cost_function(theta, x, y):
    # Computes the cost function for all the training samples
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(
        y * np.log(probability(theta, x)) + (1 - y) * np.log(
            1 - probability(theta, x)))
    return total_cost

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta,   x)) - y)

In [22]:
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta,
                  fprime=gradient,args=(x, y.flatten()))
    return opt_weights[0]


In [23]:
dfs = np.array_split(df, 2, axis=0)


In [24]:
dfs[0].head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.619995,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.690000,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.660004,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.500000,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798279,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.989998,0.0


In [25]:
dfs[1].head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
142404,84693.0,-0.439952,0.683758,1.225814,0.639113,0.716765,0.089295,0.657718,0.034213,-0.472679,...,0.127443,0.588590,-0.145728,-0.322900,-0.299744,-0.310295,0.248317,-0.011493,19.000000,0.0
142405,84694.0,-4.868108,1.264420,-5.167885,3.193648,-3.045621,-2.096166,-6.445610,2.422536,-3.214055,...,1.269205,0.057657,0.629307,-0.168432,0.443744,0.276539,1.441274,-0.127944,12.310000,1.0
142406,84695.0,1.013114,-0.334412,1.305208,0.837406,-1.126833,-0.064321,-0.594753,0.147737,0.536360,...,0.017079,0.112210,-0.016084,0.595033,0.201073,0.278215,0.007457,0.030762,66.599998,0.0
142407,84695.0,0.969231,-0.233554,0.238473,0.145793,-0.545741,-0.970680,0.347393,-0.209522,-0.342571,...,-0.362820,-1.417272,0.162136,0.541628,-0.079465,0.268702,-0.101237,0.028234,141.000000,0.0
142408,84696.0,-0.856523,1.080875,1.866956,1.729941,-0.161741,0.028789,0.401787,0.043774,-0.213916,...,0.007365,0.077392,-0.221906,0.394141,0.237225,-0.080102,-0.291408,0.092140,2.600000,0.0


In [26]:
def train(dfs,idx):
    X = dfs[idx].iloc[:, :-1]
    y = dfs[idx].iloc[:, -1]
    X = np.c_[np.ones((X.shape[0], 1)), X]
    y = y[:, np.newaxis]
    theta = np.zeros((X.shape[1], 1))
    parameters = fit(X, y, theta)
    parameters_list.append(parameters)
    print(parameters)

In [27]:
###Main function
import _thread
import time
from scipy.optimize import fmin_tnc

In [38]:
from datetime import datetime
start_time = datetime.now()
try:
   _thread.start_new_thread( train, (dfs, 0, ) )
   _thread.start_new_thread( train, (dfs, 1, ) )
except:
   print("Error: unable to start thread")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.000204


/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/3292107250.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]
  NIT   NF   F                       GTG
    0    1  6.931471805599452E-01   6.82255285E+08
tnc: fscale = 3.82848e-05
  NIT   NF   F                       GTG
    0    1  6.931471805599453E-01   4.68062120E+09
tnc: fscale = 1.46167e-05
    1    4  1.416884071488492E-01   2.79301028E+08
    1    5  3.859879429491809E-02   6.81800385E+00
tnc: fscale = 0.382976
    2    8  1.315588611488832E-02   2.32847373E+04
tnc: fscale = 0.00655336
    2   10  3.615922561994345E-02   1.59182688E+03
/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/2619135858.py:5: RuntimeWarning: divide by zero encountered in log
  y * np.log(probability(theta, x)) + (1 - y) * np.log(
    3   14  1.265216832218150E-02   1

[-1.31885329e-03 -1.30658018e-04 -5.18832778e-04  4.44252817e-06
 -2.39610087e-03  4.79136859e-04 -6.76404671e-04 -3.70667840e-04
 -8.49277268e-04  2.92596850e-04 -1.15038312e-03 -7.04101728e-04
 -1.62664289e-04  9.73616337e-05 -7.42568398e-04 -1.73842909e-03
 -1.97986115e-05 -6.45434931e-04 -1.43378537e-03 -2.82814436e-04
  1.51296738e-04  1.46305304e-05  2.58213915e-04  1.78477335e-04
  3.10131081e-05 -5.40487666e-05 -1.05776719e-04 -7.51076947e-08
  4.14591264e-05  1.37484985e-05 -3.33135882e-02]
[-1.51438827e-05 -5.17540730e-05 -2.36420953e-04  1.81518799e-04
 -3.81824052e-04  2.69405198e-04 -1.51183816e-04 -7.58372634e-05
 -3.14903697e-04 -2.37480896e-05 -1.85039600e-04 -3.27928014e-04
  1.93730226e-04 -3.10429749e-04 -3.73300142e-05 -4.13684316e-04
 -3.91648176e-06 -2.17581883e-04 -3.36175164e-04 -1.08126084e-04
  3.86853504e-05  3.40823813e-05  2.86230582e-06  1.93505567e-05
  1.45390266e-06 -5.76357257e-06 -3.30481748e-06  2.10369654e-06
 -1.55927041e-05  3.36753633e-06  2.6454

In [35]:
final_parameters = (parameters_list[0]+parameters_list[1])/2
print(final_parameters)

[-6.66998588e-04 -9.12060454e-05 -3.77626866e-04  9.29806636e-05
 -1.38896246e-03  3.74271028e-04 -4.13794243e-04 -2.23252551e-04
 -5.82090482e-04  1.34424380e-04 -6.67711358e-04 -5.16014871e-04
  1.55329688e-05 -1.06534058e-04 -3.89949206e-04 -1.07605670e-03
 -1.18575466e-05 -4.31508407e-04 -8.84980265e-04 -1.95470260e-04
  9.49910441e-05  2.43564558e-05  1.30538110e-04  9.89139458e-05
  1.62335054e-05 -2.99061696e-05 -5.45407681e-05  1.01429442e-06
  1.29332112e-05  8.55801743e-06 -1.65245226e-02]


In [36]:
def predict(x):
    theta = final_parameters[:, np.newaxis]
    return probability(theta, x)
def accuracy(x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x) >= 
                         probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = np.mean(predicted_classes == actual_classes)
    return accuracy * 100

In [37]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X = np.c_[np.ones((X.shape[0], 1)), X]
y = y[:, np.newaxis]
theta = np.zeros((X.shape[1], 1))
accuracy(X, y.flatten())

/var/folders/kw/j6znb2j11h3g511fp04pyc6m0000gn/T/ipykernel_42345/8894125.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


99.82725143693798